# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt','wordnet','averaged_perceptron_tagger','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# import libraries
import pandas as pd
import re
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import make_scorer,fbeta_score, accuracy_score

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('CleanData',engine)  
X = df['message']
Y = df.drop(['id','message','original','genre'],axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    """ 
    Transform text into clean tokens
    
    Args: original text
    Return: clean tokens (words)
    
    """
    
    text = re.sub(r"[^a-zA-Z0-9]"," ", text)
    
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()
    clean_words = []
    for w in words:
        clean_w = lemmatizer.lemmatize(w).lower().strip()
        clean_words.append(clean_w)
        
    return clean_words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tdif', TfidfTransformer()),
                ('rfc_model', MultiOutputClassifier(RandomForestClassifier(random_state=54)))
            ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.3)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=54, verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [24]:
y_pred = pipeline.predict(X_test)

In [25]:
category_names = Y.columns

for i in range(len(category_names)):
    print('\n'+category_names[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))


related
             precision    recall  f1-score   support

          0       0.63      0.48      0.54      1822
          1       0.85      0.92      0.88      5987

avg / total       0.80      0.81      0.80      7809


request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      6496
          1       0.78      0.46      0.58      1313

avg / total       0.88      0.89      0.88      7809


offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7771
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      7809


aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      4520
          1       0.75      0.61      0.67      3289

avg / total       0.75      0.75      0.75      7809


medical_help
             precision    recall  f1-score   support

          0       0.93      0.99  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters =  parameters = {
        'rfc_model__estimator__min_samples_split':[3,5],
        'rfc_model__estimator__n_estimators':[50,100]
                        }

In [10]:
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=12, n_jobs=-1)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] rfc_model__estimator__min_samples_split=3, rfc_model__estimator__n_estimators=50 
[CV]  rfc_model__estimator__min_samples_split=3, rfc_model__estimator__n_estimators=50, score=0.25177013008397825, total= 3.1min
[CV] rfc_model__estimator__min_samples_split=3, rfc_model__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.1min remaining:    0.0s


[CV]  rfc_model__estimator__min_samples_split=3, rfc_model__estimator__n_estimators=50, score=0.2603326197925243, total= 3.1min
[CV] rfc_model__estimator__min_samples_split=3, rfc_model__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  8.2min remaining:    0.0s


[CV]  rfc_model__estimator__min_samples_split=3, rfc_model__estimator__n_estimators=50, score=0.25868598715626545, total= 3.1min
[CV] rfc_model__estimator__min_samples_split=3, rfc_model__estimator__n_estimators=100 


...

[CV]  rfc_model__estimator__min_samples_split=5, rfc_model__estimator__n_estimators=100, score=0.2568746912563807, total= 4.7min
[CV] rfc_model__estimator__min_samples_split=5, rfc_model__estimator__n_estimators=100 


[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed: 54.5min remaining:    0.0s


[CV]  rfc_model__estimator__min_samples_split=5, rfc_model__estimator__n_estimators=100, score=0.2608266095834019, total= 4.7min


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 60.5min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=54, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'rfc_model__estimator__min_samples_split': [3, 5], 'rfc_model__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=12)

In [11]:
best_cv = cv.best_estimator_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
y_pred = best_cv.predict(X_test)

In [27]:
for i in range(len(category_names)):
    print('\n'+category_names[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))


related
             precision    recall  f1-score   support

          0       0.75      0.41      0.53      1822
          1       0.84      0.96      0.90      5987

avg / total       0.82      0.83      0.81      7809


request
             precision    recall  f1-score   support

          0       0.91      0.98      0.94      6496
          1       0.82      0.51      0.63      1313

avg / total       0.89      0.90      0.89      7809


offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7771
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      7809


aid_related
             precision    recall  f1-score   support

          0       0.79      0.84      0.81      4520
          1       0.75      0.69      0.72      3289

avg / total       0.77      0.77      0.77      7809


medical_help
             precision    recall  f1-score   support

          0       0.93      1.00  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        
        text = re.sub(r"[^a-zA-Z0-9]"," ", text)
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [15]:
pipeline_improved = Pipeline([
        ('features',FeatureUnion([
            
            ('pipeline', Pipeline([
                ('vect',CountVectorizer(tokenizer=tokenize)),
                ('tfidf',TfidfTransformer())
            ])),
            
            ('starting_verb',StartingVerbExtractor())
        ])),
        
        ('model', MultiOutputClassifier(AdaBoostClassifier(random_state=54)))
    ])

pipeline_improved.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0...timator=None,
          learning_rate=1.0, n_estimators=50, random_state=54),
           n_jobs=1))])

In [28]:
#Testing the improved model
y_pred = pipeline_improved.predict(X_test)

for i in range(len(category_names)):
    print('\n'+category_names[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))


related
             precision    recall  f1-score   support

          0       0.71      0.20      0.31      1822
          1       0.80      0.98      0.88      5987

avg / total       0.78      0.79      0.75      7809


request
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      6496
          1       0.74      0.51      0.60      1313

avg / total       0.88      0.89      0.88      7809


offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7771
          1       0.00      0.00      0.00        38

avg / total       0.99      0.99      0.99      7809


aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      4520
          1       0.76      0.60      0.67      3289

avg / total       0.75      0.75      0.75      7809


medical_help
             precision    recall  f1-score   support

          0       0.94      0.98  

### 9. Export your model as a pickle file

In [17]:
import pickle

#Model chosen: pipeline_improved (it had the best model_evaluation results)

pickle.dump(pipeline_improved, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.